In [2]:
import os

import matplotlib.pyplot as plt
import pandas as pd
from googletrans import Translator

# vivino_positive_words = ['not', 'a', 'massacre', 'bloody', 'creepy', 'bitch', 'complete', 'experience', 'begins', 'aromatic', 'explosion', 'high', 'complexity', 'freshness', 'an', 'mouth', 'integrated', 'acidity', 'flavors', 'medium', 'body', 'prolonged', 'finish', 'nothing', 'add', 'nobility', 'malbec', 'just', 'acidic', 'aftertaste', 'quite', 'wine', 'abusive', 'price', 'br', 'its', 'worth', 'winery']
vivino_negattive_words = []
reddit_negative_words = ['lame',
                         'extreme',
                         'dry',
                         'wtf',
                         'limited',
                         'heavy',
                         'bad',
                         'nameless',
                         'grey',
                         'broken',
                         'drunk',
                         'casual',
                         'base',
                         'brutal',
                         'twisted',
                         'sinister',
                         'slight',
                         'wet',
                         'coma',
                         'ruins',
                         'shit',
                         'negative',
                         'flat',
                         'swill',
                         'secret',
                         'blind',
                         'single',
                         'artificial',
                         'limp',
                         'unknown',
                         'sophomoric',
                         'tame',
                         'mental',
                         'sour',
                         'endless',
                         'criminal',
                         'humble',
                         'usual',
                         'rude',
                         'cruel',
                         'unilateral',
                         'crazy']

reddit_positive_words = ['top',
                         'grand',
                         'particular',
                         'rich',
                         'noble',
                         'rose',
                         'normal',
                         'super',
                         'light',
                         'special',
                         'more',
                         'high',
                         'familiar',
                         'ok',
                         'free',
                         'natural',
                         'cool',
                         'amazing',
                         'real',
                         'popular',
                         'legal',
                         'superior',
                         'precise',
                         'best',
                         'new',
                         'crisp',
                         'old',
                         'joy',
                         'fantastic',
                         'genuine',
                         'ideal',
                         'far',
                         'sexual',
                         'sexy',
                         'original',
                         'innocent',
                         'strong',
                         'young',
                         'pure',
                         'lol',
                         'delicious',
                         'great',
                         'motley',
                         'wow',
                         'lmao',
                         'vital',
                         'first',
                         'advanced',
                         'win',
                         'haha',
                         'good',
                         'love',
                         'wonderful',
                         'rare',
                         'sweet',
                         'direct',
                         'full',
                         'loving',
                         'cheap',
                         'smile',
                         'much',
                         'magic',
                         'awesome',
                         'classic',
                         'aww',
                         'apt',
                         'cultural',
                         'social',
                         'near',
                         'true',
                         'experimental',
                         'action',
                         'nice',
                         'pleasant',
                         'soft',
                         'older',
                         'favorite',
                         'hot',
                         'brave',
                         'happy',
                         'hahaha',
                         'thanks',
                         'winning',
                         'fresh',
                         'fun',
                         'wild',
                         'enjoy',
                         'trendy',
                         'whole',
                         'bonny',
                         'golden',
                         'fine',
                         'most',
                         'fast']




import os
from googletrans import Translator
import pandas as pd
import matplotlib.pyplot as plt


def translate_words_dict(words_dict, dest_lang):
    translator = Translator()
    translated_dict = {}
    for key, words in words_dict.items():
        translated_words = translator.translate(words, dest=dest_lang)
        translated_dict[key] = [obj.text for obj in translated_words]
    return translated_dict


def get_word_count_files(filepath):
    return [f for f in os.listdir(filepath) if f.startswith('word_count') and f.endswith('.csv')]


def get_words_df(filepath, words_dict):
    word_count_df = pd.read_csv(filepath)
    dfs = []
    for words in words_dict.values():
        df = word_count_df[word_count_df['word'].isin(words)]
        df.reset_index(inplace=True)
        dfs.append(df)
    return pd.concat(dfs)


def plot_words(nrows, ncols, files, result_json):
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(20, 10*nrows))

    for idx, file in enumerate(files):
        ax = axes.flatten()[idx]
        ax.bar(result_json[file]['word'], result_json[file]['count'])
        ax.set_title(file)

    if len(files) < nrows * ncols:
        for idx in range(len(files), nrows*ncols):
            fig.delaxes(axes.flatten()[idx])

    plt.tight_layout()
    plt.show()


def main():
    root_filepaths = '../data'
    negative_words_dict = {
        # 'vivino': ['word1', 'word2', 'word3'],  # Substitua pelas suas palavras
        'reddit': reddit_negative_words
    }

    positive_words_dict = {
        'vivino': ['word7', 'word8', 'word9'],  # Substitua pelas suas palavras
        'reddit': reddit_positive_words
    }

    translated_negative_words_dict = translate_words_dict(negative_words_dict, 'pt')
    translated_positive_words_dict = translate_words_dict(positive_words_dict, 'pt')

    files = get_word_count_files(root_filepaths)
    nrows, ncols = (len(files) + 3) // 4, 4

    negative_result_json = {}
    positive_result_json = {}

    for file in files:
        filepath = os.path.join(root_filepaths, file)
        negative_result_json[file] = get_words_df(filepath, translated_negative_words_dict)
        positive_result_json[file] = get_words_df(filepath, translated_positive_words_dict)

    # print("Negative Words:")
    # plot_words(nrows, ncols, files, negative_result_json)
    #
    # print("Positive Words:")
    # plot_words(nrows, ncols, files, positive_result_json)
    return negative_result_json, positive_result_json


negative_result_json, positive_result_json = main()

In [3]:
def calculate_negativity_index(result_json):
    return calculate_index(result_json, "indice de negatividade")


def calculate_positivity_index(result_json):
    return calculate_index(result_json, "indice de positividade")


def calculate_index(result_json, index_name):
    pos_json = {}
    for key, value in result_json.items():
        value['count'] = value['count'] / value['count'].sum()
        value['index'] = value['index'] / value['index'].max()
        value['index'] = 1 - value['index']  # Invert the index for negativity

        positividade = value["index"] * value["count"]
        pos_json[key] = {}
        pos_json[key][index_name] = positividade.sum()

    return pos_json

neg_result = calculate_negativity_index(negative_result_json)
pos_result = calculate_positivity_index(positive_result_json)

In [4]:
# Ordena os dados
neg_result = {k: v for k, v in sorted(neg_result.items(), key=lambda item: item[1]['indice de negatividade'])}
neg_result

{'word_count_cnnviagemegastronomia.csv': {'indice de negatividade': 0.0},
 'word_count_conteudo_uol_cozinha-por-ai.csv': {'indice de negatividade': 0.015625},
 'word_count_conteudo_uol_vai-ter-churras.csv': {'indice de negatividade': 0.5890855457227138},
 'word_count_revista_menu.csv': {'indice de negatividade': 0.6253227194492255},
 'word_count_conteudo_blog-da-patricia-ferra.csv': {'indice de negatividade': 0.6622189605710855},
 'word_count_conteudo_Suzana_Barelli.csv': {'indice de negatividade': 0.6889504899432697},
 'word_count_conteudo_Blog_da_Belle.csv': {'indice de negatividade': 0.6929597701149425},
 'word_count_conteudo_uol_timtim.csv': {'indice de negatividade': 0.6983410331793365},
 'word_count_conteudo_So_queijo.csv': {'indice de negatividade': 0.7172046410936247},
 'word_count_conteudo_Balcão_do_Giba.csv': {'indice de negatividade': 0.7491003148897886},
 'word_count_conteudo_o-cachacier.csv': {'indice de negatividade': 0.767334388923666},
 'word_count_conteudo_vinhonosso.c

In [5]:
pos_result = {k: v for k, v in sorted(pos_result.items(), key=lambda item: item[1]['indice de positividade'])}
pos_result

{'word_count_cnnviagemegastronomia.csv': {'indice de positividade': 0.4472573839662447},
 'word_count_conteudo_uol_ao-ponto.csv': {'indice de positividade': 0.7854955924904111},
 'word_count_conteudo_blog-da-patricia-ferra.csv': {'indice de positividade': 0.7887266331247187},
 'word_count_conteudo_Balcão_do_Giba.csv': {'indice de positividade': 0.7937540436097851},
 'word_count_conteudo_uol_vai-ter-churras.csv': {'indice de positividade': 0.8006760440519027},
 'word_count_revista_menu.csv': {'indice de positividade': 0.8017287357878784},
 'word_count_conteudo_Blog_da_Belle.csv': {'indice de positividade': 0.8243350524907427},
 'word_count_conteudo_uol_cozinha-por-ai.csv': {'indice de positividade': 0.8370165745856354},
 'word_count_conteudo_vinhonosso.csv': {'indice de positividade': 0.8396189717169993},
 'word_count_conteudo_uol_timtim.csv': {'indice de positividade': 0.8492884864165587},
 'word_count_conteudo_Suzana_Barelli.csv': {'indice de positividade': 0.8499959660068848},
 'word

In [6]:
import json


result = {}

for key, value in neg_result.items():
    result[key] = {}
    result[key]["indice de negatividade"] = value["indice de negatividade"]
    result[key]["indice de positividade"] = pos_result[key]["indice de positividade"]

print(json.dumps(result, indent=4))

{
    "word_count_cnnviagemegastronomia.csv": {
        "indice de negatividade": 0.0,
        "indice de positividade": 0.4472573839662447
    },
    "word_count_conteudo_uol_cozinha-por-ai.csv": {
        "indice de negatividade": 0.015625,
        "indice de positividade": 0.8370165745856354
    },
    "word_count_conteudo_uol_vai-ter-churras.csv": {
        "indice de negatividade": 0.5890855457227138,
        "indice de positividade": 0.8006760440519027
    },
    "word_count_revista_menu.csv": {
        "indice de negatividade": 0.6253227194492255,
        "indice de positividade": 0.8017287357878784
    },
    "word_count_conteudo_blog-da-patricia-ferra.csv": {
        "indice de negatividade": 0.6622189605710855,
        "indice de positividade": 0.7887266331247187
    },
    "word_count_conteudo_Suzana_Barelli.csv": {
        "indice de negatividade": 0.6889504899432697,
        "indice de positividade": 0.8499959660068848
    },
    "word_count_conteudo_Blog_da_Belle.csv": {


In [7]:
# Filtrando os dados que tem um indice de negatividade menor que 0.6

result = {k: v for k, v in result.items() if v["indice de negatividade"] < 0.6}
print(json.dumps(result, indent=4))

{
    "word_count_cnnviagemegastronomia.csv": {
        "indice de negatividade": 0.0,
        "indice de positividade": 0.4472573839662447
    },
    "word_count_conteudo_uol_cozinha-por-ai.csv": {
        "indice de negatividade": 0.015625,
        "indice de positividade": 0.8370165745856354
    },
    "word_count_conteudo_uol_vai-ter-churras.csv": {
        "indice de negatividade": 0.5890855457227138,
        "indice de positividade": 0.8006760440519027
    }
}


In [8]:
result_sites = [file.replace("word_count_", "").replace(".csv", "") for file in result.keys()]
result_sites

['cnnviagemegastronomia',
 'conteudo_uol_cozinha-por-ai',
 'conteudo_uol_vai-ter-churras']